In [1]:
'''
Graph and Loss visualization using Tensorboard.
'''

from __future__ import print_function

import tensorflow as tf


In [2]:
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# Parameters
learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_epoch = 1
logs_path = '/users/ektas/tensorboardlogs_mnist/'

In [4]:
# tf Graph Input
# mnist data image of shape 28*28=784
x = tf.placeholder(tf.float32, [None, 784], name='InputData')
# 0-9 digits recognition => 10 classes
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

In [5]:
# Set model weights
W = tf.Variable(tf.zeros([784, 10]), name='Weights')
b = tf.Variable(tf.zeros([10]), name='Bias')

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
# Construct model and encapsulating all ops into scopes, making
# Tensorboard's Graph visualization more convenient
with tf.name_scope('Model'):
    # Model
    pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax
with tf.name_scope('Loss'):
    # Minimize error using cross entropy
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

Instructions for updating:
Use tf.cast instead.


In [7]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [8]:
# Create a summary to monitor cost tensor
tf.summary.scalar("loss", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("accuracy", acc)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()


In [9]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_epoch == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")
    summary_writer.close()
    # Test model
    # Calculate accuracy
    print("Accuracy:", acc.eval({x: mnist.test.images, y: mnist.test.labels}))

Epoch: 0001 cost= 1.183992049
Epoch: 0002 cost= 0.665430229
Epoch: 0003 cost= 0.552850635
Epoch: 0004 cost= 0.498637856
Epoch: 0005 cost= 0.465512632
Epoch: 0006 cost= 0.442596177
Epoch: 0007 cost= 0.425510912
Epoch: 0008 cost= 0.412190542
Epoch: 0009 cost= 0.401421090
Epoch: 0010 cost= 0.392424267
Epoch: 0011 cost= 0.384809088
Epoch: 0012 cost= 0.378168615
Epoch: 0013 cost= 0.372408052
Epoch: 0014 cost= 0.367327515
Epoch: 0015 cost= 0.362700881
Epoch: 0016 cost= 0.358597667
Epoch: 0017 cost= 0.354848301
Epoch: 0018 cost= 0.351440375
Epoch: 0019 cost= 0.348315746
Epoch: 0020 cost= 0.345444180
Epoch: 0021 cost= 0.342683879
Epoch: 0022 cost= 0.340220161
Epoch: 0023 cost= 0.337912483
Epoch: 0024 cost= 0.335743537
Epoch: 0025 cost= 0.333697582
Optimization Finished!
Accuracy: 0.9141
